In [2]:
import sqlite3 as sql
import os
import csv
from sqlite3 import Error
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import ast

import pygad
import numpy as np
import matplotlib.pyplot as plt
import math
import csv
import tqdm
from scipy.signal import find_peaks, peak_prominences, peak_widths, savgol_filter
from scipy.optimize import curve_fit
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource, HoverTool

import shutil
import os

In [3]:
try:

  # Connect to database
  conn=sql.connect('minspec.db')


 # To view table data in table format
  print("******Mineral Table Data*******")
  cur = conn.cursor()
  cur.execute('''SELECT * FROM minerals''')
  rows = cur.fetchall()
   
  #for row in rows:
  #    print(row)

 # Export data into CSV file
  print("Exporting data into CSV............")
  cursor = conn.cursor()
  cursor.execute("select * from minerals")
  with open("RAWmineral_data.csv", "w") as csv_file:
      csv_writer = csv.writer(csv_file, delimiter="\t")
      csv_writer.writerow([i[0] for i in cursor.description])
      csv_writer.writerows(cursor)

  dirpath = os.getcwd() + "/RAWmineral_data.csv"
  print("Data exported Successfully into {}".format(dirpath))

except Error as e:
  print(e)

# Close database connection
finally:
  conn.close()

******Mineral Table Data*******
Exporting data into CSV............
Data exported Successfully into /Users/mrecarte/Desktop/Ocean_World_Spectra-main/RAWmineral_data.csv


In [72]:
with open("RAWmineral_data.csv", 'r') as temp_f:
# get number of columns in each line
    col_count = [ len(l.split("\t")) for l in temp_f.readlines() ]

# Generate column names  (names will be 0, 1, 2, ..., maximum columns - 1)
column_names = [i for i in range(0, max(col_count))]

# Read csv
df = pd.read_csv("RAWmineral_data.csv", header=None, delimiter="\t", 
names=column_names)
col_names = list(df.iloc[0])
df = pd.read_csv("RAWmineral_data.csv", header=None, delimiter="\t", names=col_names)
df.drop(0, inplace=True); df.reset_index(drop=True, inplace=True)

# Convert reflectances to lists of floats
df['reflectances'] = df['reflectances'].apply(lambda x: list(map(float, eval(x))) if x != 'None' else None)

# Calculate the mean reflectances for each group
mean_reflectances = df.groupby(['name', 'sampleID'])['reflectances'].apply(
    lambda x: np.mean(x.tolist(), axis=0) if x.notnull().any() else None
)

# Remove duplicate rows based on 'name' and 'sampleID'
df.drop_duplicates(subset=['name', 'sampleID'], keep='first', inplace=True)

# Overwrite the 'reflectances' column with the mean reflectances
df['reflectances'] = df.apply(
    lambda row: mean_reflectances[(row['name'], row['sampleID'])] if (row['name'], row['sampleID']) in mean_reflectances.index else None,
    axis=1
)

# Remove special characters from the "chemical_formula" column
df["chemical_formula"] = df["chemical_formula"].str.replace(r"[#&_\s]", "")

# Convert reflectances to comma-separated strings
df['reflectances'] = df['reflectances'].apply(lambda x: ', '.join(map(str, x)) if x is not None else None)

# Save the DataFrame to a CSV file
df.to_csv('output.csv', index=False)


df


,name,chemical_formula,sampleID,sample_purity,wavelengths,reflectances
0,Lizardite,Mg3Si2O5(OH)4,NMNHR4687.b_165um,NIC4aa,"[1.3447871, 1.345136, 1.3454852, 1.3458345, 1....","0.54064333, 0.54122853, 0.54210639, 0.54196006..."
1,Bloedite,Na2Mg(SO4)2*4H2O,GDS147,NIC4aau,"[1.5283473, 1.528798, 1.529249, 1.5297003, 1.5...","0.15803361, 0.15800314, 0.1580946, 0.15824707,..."
2,Illite,"(K,H3O)(Al,Mg,Fe)2(Si,Al)4O10[(OH)2,H2O]",IMt-1.b_lt2um,NIC4aa,"[1.3894811, 1.3898535, 1.3902262, 1.3905991, 1...","0.33545622, 0.33515427, 0.33545622, 0.33515427..."
3,Jarosite,K(Fe+3)3(SO4)2(OH)6,GDS98_K_90C_Syn,NIC4aa,"[1.4340675, 1.4344643, 1.4348613, 1.4352585, 1...","0.7534914, 0.75223202, 0.75242579, 0.75184453,..."
4,Anhydrite,CaSO4,GDS42_lt250um,NIC4aa,"[1.7217383, 1.7223103, 1.7228827, 1.7234554, 1...","0.77212828, 0.77273488, 0.77288651, 0.7733416,..."
...,...,...,...,...,...,...
107,Water_Ice,H2O,300micron_irr_80K,Not given,"[0.45, 0.5, 0.55, 0.6, 0.65, 0.700000000000000...","0.87419, 0.89482, 0.91041, 0.90928, 0.90426, 0..."
109,Water_Ice,H2O,300micron_irr_100K,Not given,"[0.45, 0.5, 0.55, 0.6, 0.65, 0.700000000000000...","0.98113, 0.9378, 0.9098, 0.90286, 0.90069, 0.8..."
110,Water_Ice,H2O,1060micron_120K,Not given,"[0.45, 0.5, 0.55, 0.6, 0.65, 0.700000000000000...","0.75039, 0.77018, 0.77242, 0.76489, 0.76332, 0..."
116,Water_Ice,H2O,680micron_140K,Not given,"[0.45, 0.5, 0.55, 0.6, 0.65, 0.700000000000000...","0.80806, 0.84044, 0.83142, 0.82549, 0.81313, 0..."


In [73]:
df['reflectances'] = df['reflectances'].apply(lambda x: ast.literal_eval(x))
df['wavelengths'] = df['wavelengths'].apply(lambda x: ast.literal_eval(x))

df['wavelengths'] = df['wavelengths'].apply(lambda x: np.array(x))
df['reflectances'] = df['reflectances'].apply(lambda x: np.array(x))

df['wavenumbers'] = df['wavelengths'].apply(lambda x: 1/x*(10**6)/(10**2))

In [74]:
dataframe = df
dataframe = dataframe[0:41]

In [87]:

for index, row in dataframe.iterrows():
    mineral_name = row['name']
    wavenumbers = row['wavenumbers']
    reflectances = row['reflectances']
    
    # Plot original data
    # plt.figure(figsize=(8, 4))
    # plt.plot(wavenumbers, reflectances, label='Original Reflectance')
    # plt.xlabel('Wavelength (microns)')
    # plt.ylabel('Reflectance')
    # plt.title(f'Original Reflectance Spectrum for {mineral_name}')
    # plt.legend()
    # plt.show()

    # Process reflectance data
    data = np.column_stack((wavenumbers, reflectances))
    v = data[:, 0]
    #R = data[:, 1] / 100
    R = data[:, 1]
    
    # Cropping criteria for wavenumbers (in microns)
    min_wavelength = 400
    max_wavelength = 1800
    
    # Find indices that satisfy the cropping criteria
    valid_indices = np.where((v >= min_wavelength) & (v <= max_wavelength))
    
    # Apply cropping to data
    cropped_v = v[valid_indices]
    cropped_R = R[valid_indices]

    # Plot cropped data
    # plt.figure(figsize=(8, 4))
    # plt.plot(cropped_v, cropped_R, 'k', label="Cropped Reflectance")
    # plt.xlabel('Wavenumber (cm^-1)')
    # plt.ylabel("Reflectance")
    # plt.title(f'Cropped Reflectance Spectrum for {mineral_name}')
    # plt.legend()
    # plt.show()
    

    def fresnel_sbu(n,k, viewing_angle): #theta = viewing_angledeg
        u1 = np.square(n)-np.square(k)-np.square(np.sin(viewing_angle*math.pi/180))
        u2 = np.sqrt(np.square(u1) + (4*np.multiply(np.square(n),np.square(k))))
        u = np.sqrt((u1+u2)/2)

        v = np.sqrt((-u1 + u2)/2)


        R_par_num = np.square(np.cos(viewing_angle*math.pi/180)-u)+ np.square(v)
        R_par_denom = np.square(np.cos(viewing_angle*math.pi/180)+u)+ np.square(v)
        R_par = np.divide(R_par_num,R_par_denom)

        R_perp_num = np.square(((np.square(n)-np.square(k))*np.cos(viewing_angle*math.pi/180))-u) + np.square((2*np.cos(viewing_angle*math.pi/180)*np.multiply(n,k))-v)
        R_perp_denom = np.square(((np.square(n)-np.square(k))*np.cos(viewing_angle*math.pi/180))+u) + np.square((2*np.cos(viewing_angle*math.pi/180)*np.multiply(n,k))+v)
        R_perp = np.divide(R_perp_num,R_perp_denom)

        R_sbu = (R_perp+R_par)/2

        return R_sbu
    
    def dispersion_model(nu,gamm,fourpr,epsil,v):
        # dispersion_model: Spitzer and Kleinman dispersion model
        # v: frequency list
        # gamm: damping parameter
        # fourpr: bandwidth parameter
        # epsil: bulk dialectric constant

        # make v be Mx1
        v = v[:,None]

        # make nu, fourpr, gamm be 1xN 
        nu = nu[:,None].T
        fourpr = fourpr[:,None].T 
        gamm = gamm[:,None].T 

        v = np.tile(v,(1,len(nu)))
        nu = np.tile(nu,(len(v),1))
        fourpr = np.tile(fourpr,(len(v),1))
        gamm = np.tile(gamm,(len(v),1))

        a = np.square(nu)-np.square(v)
        denom = np.square(a) + np.square(nu * gamm * v)
        alpha = np.divide((fourpr*np.square(nu)*a),denom)
        bet = np.divide(((fourpr/2) * np.square(nu) * gamm * nu * v),denom)
        theta = epsil + np.sum(alpha,axis=1)
        phi = np.sum(bet,axis=1)
        a2 = np.sqrt(np.square(theta) + (4*np.square(phi)))
        n = np.sqrt((theta+a2)/2)
        k = np.divide(phi,n)

        return n, k
    
    def calculate_rnk(coef,v, viewing_angle):
        # disp_model_wrap: wrapper for dispersion model

        N = int((len(coef)-1)/3)
        #     if isinstance(N, int): 
        #         print('ERROR: disp_model_wrap_sbu: bad length for coefficient list!')
        
        nu = coef[0:N]
        gamm = coef[N:N+N]
        fourpr = coef[N+N:N+N+N]
        epsil = coef[N+N+N]

        n,k = dispersion_model(nu,gamm,fourpr,epsil,v)
        r = fresnel_sbu(n,k, viewing_angle)
        print(n)
        print(k)

        return r,n,k
    
    output_notebook()

    df = {'wavelength':v,
        'reflectance':R,
        'sav_gol_R':savgol_filter(R, 21,3),
        'sav_gol_R_diff':savgol_filter(R,21,3, deriv=2)}
    
    
    source = ColumnDataSource(df)
    p = figure(width=500, height=300, title="measured reflectance", x_axis_label="wavenumber", y_axis_label="reflectance")
    p.line('wavelength','reflectance', source = source, legend_label="R_measured", line_width=1, color='black')
    p.line('wavelength','sav_gol_R', source = source, legend_label="R_smooth", line_width=2, color='red')

    p.add_tools(HoverTool(show_arrow=False, line_policy='next', tooltips=[
    ('v','@wavelength{0.000}'),
    ('R', '@reflectance'),
    ("index", "$index")
    ], 
        formatters={
            'v': 'printf',
            'R' : 'printf',
        }))

    show(p)

    p = figure(width=500, height=300, title="Smoothed Reflectance 1st derivative", x_axis_label="wavenumber", y_axis_label="reflectance")
    p.line('wavelength','sav_gol_R_diff', source = source, legend_label="R_smooth_diff", line_width=1, color='red')

    p.add_tools(HoverTool(show_arrow=False, line_policy='next', tooltips=[
    ('v','@wavelength{0.000}'),
    ('R', '@reflectance'),
    ("index", "$index")
    ],
        formatters={
        'v': 'printf',
        'R' : 'printf',
        }))
    
    show(p)

    R_smoothed = savgol_filter(R, 21,3) 

    spectra_range = {'R1': range(25), 'R2': range(38,111), 'R3':range(110, 164), 'R4':range(164, 190), 'R5':range(190, 223), 'R6': range(225, len(v))}

    def fitness_func(ga_instance, solution, solution_idx):
        global v
        global R_smoothed
    
        r, n, k = calculate_rnk(solution, v, viewing_angle)
    
        fitness = (1-chisq(r, R_smoothed)) * 100  # calculating euclidean distance
    
        return fitness
    
    def chisq(modelled, measured):
        return np.round(np.sum(np.square(modelled-measured)/measured),3)
    
    def ga_optical_constants(gene_space): 
        def on_generation_progress(ga):
        #         print("Generation", ga.generations_completed)
        #         print("Generation shape", ga.population.shape)
        #         print(ga.population)
            pbar.update(1)
            
        def parent_selection_func(fitness, num_parents, ga_instance):
            #         print("Generation", ga_instance.generations_completed)
            fitness_sorted = sorted(range(len(fitness)), key=lambda k: fitness[k])
            fitness_sorted.reverse()
            #         print("fitness sorted", fitness_sorted)
        
            oscillators_num =  int((ga_instance.population.shape[1]-1)/3)

            parents = np.empty((num_parents, ga_instance.population.shape[1]))
        
            parent_num = 0
        
            for fs in fitness_sorted:
                osci_diff = np.ediff1d(ga_instance.population[fs, :oscillators_num])
                
                if parent_num < num_parents:
                    if np.all(np.abs(osci_diff) > 10):
                        parents[parent_num, :] = ga_instance.population[fs, :].copy()
                        parent_num+=1
                else:
                    break

            return parents, np.array(fitness_sorted[:num_parents])

            
        num_generations=10000
        sol_per_pop=100
        num_parents_mating=10
        crossover_probability = 0.15
        mutation_percent_genes=(3,2)

    
        with tqdm.tqdm(total=num_generations) as pbar:
            ga_instance = pygad.GA(fitness_func=fitness_func,
                               num_generations=num_generations,
                               sol_per_pop=sol_per_pop,
                               num_genes=len(gene_space),
                               num_parents_mating=num_parents_mating,
                               gene_space = gene_space,
                               gene_type=float,
#                                parent_selection_type='sss',
                              parent_selection_type=parent_selection_func,
                               crossover_type='two_points', 
                               crossover_probability = crossover_probability,
                               mutation_type='adaptive', 
                              mutation_percent_genes=mutation_percent_genes,
                              save_best_solutions=True,
                              save_solutions=True,
                              suppress_warnings=True,
                              allow_duplicate_genes=False,
                               stop_criteria="saturate_100",
                              on_generation=on_generation_progress)

            ga_instance.run() 

        return ga_instance

    def func_lsq_R(v, *ga_oscillation_parameters):
        # disp_model_wrap: wrapper for dispersion model

        ga_oscillation_parameters = np.array(ga_oscillation_parameters)

        N = int((len(ga_oscillation_parameters)-1)/3)

        nu = ga_oscillation_parameters[0:N]
        gamm = ga_oscillation_parameters[N:N+N]
        fourpr = ga_oscillation_parameters[N+N:N+N+N]
        epsil = ga_oscillation_parameters[N+N+N]

        n,k = dispersion_model(nu,gamm,fourpr,epsil,v)
        r = fresnel_sbu(n,k, viewing_angle)

        return r
    
    def count(sett, rangee):
        c=0   
        for val in sett:
            if  rangee[0] < val < rangee[-1]:
                c+=1       
        return c


    ga_R_mod = {}
    ga_R_res = {}
    ga_n = {}
    ga_k = {}


    ga_lsq_R_mod = {}
    ga_lsq_R_res = {}
    ga_lsq_n = {}
    ga_lsq_k = {}

    lst_folders = ['best_solution_fitness', 'optical_constants', 'oscillation_params']

    for lf in lst_folders:
        if os.path.isdir(lf):
            shutil.rmtree(lf)
            os.mkdir(lf)
        else:
            os.mkdir(lf)
            

    max_num_oscillators_R1 = 1
    max_num_oscillators_R2 = 5
    max_num_oscillators_R3 = 5
    max_num_oscillators_R4 = 1 
    max_num_oscillators_R5 = 5 
    max_num_oscillators_R6 = 0 

    osc2 = 1
    num_oscillators =2 
    total_num_oscillators = max_num_oscillators_R1 + max_num_oscillators_R2 + max_num_oscillators_R3 + max_num_oscillators_R4 + max_num_oscillators_R5 + max_num_oscillators_R6


    run = 1

    while num_oscillators < total_num_oscillators:    
        if osc2 == 1:
            lb_ga_nu = np.repeat(420,num_oscillators)
            ub_ga_nu = np.repeat(1300,num_oscillators)
            osc2 = 0
            
        else:
            lb_ga_nu = ga_solution[:num_oscillators]-10.
            ub_ga_nu = ga_solution[:num_oscillators]+10.

            if chisq_val_R1 >= 0.03:
                lb_ga_nu = np.append(lb_ga_nu, v[spectra_range['R1']][0]+10)
                ub_ga_nu = np.append(ub_ga_nu, v[spectra_range['R1']][-1]-10)      

            if chisq_val_R2 >= 0.005:
                if count(ga_solution[:num_oscillators], v[spectra_range['R2']]) < max_num_oscillators_R2:
                    lb_ga_nu = np.append(lb_ga_nu, v[spectra_range['R2']][0]+10)
                    ub_ga_nu = np.append(ub_ga_nu, v[spectra_range['R2']][-1]-10)      

            if chisq_val_R3 >= 0.005:
                if count(ga_solution[:num_oscillators], v[spectra_range['R3']]) < max_num_oscillators_R3:
                    lb_ga_nu = np.append(lb_ga_nu, v[spectra_range['R3']][0]+10)
                    ub_ga_nu = np.append(ub_ga_nu, v[spectra_range['R3']][-1]-10)
                
            if chisq_val_R4 >= 0.005:
                if count(ga_solution[:num_oscillators], v[spectra_range['R4']]) < max_num_oscillators_R4:
                    lb_ga_nu = np.append(lb_ga_nu, v[spectra_range['R4']][0]+10)
                    ub_ga_nu = np.append(ub_ga_nu, v[spectra_range['R4']][-1]-10)      

            if chisq_val_R5 >= 0.005:
                if count(ga_solution[:num_oscillators], v[spectra_range['R5']]) < max_num_oscillators_R5:
                    lb_ga_nu = np.append(lb_ga_nu, v[spectra_range['R5']][0]+10)
                    ub_ga_nu = np.append(ub_ga_nu, v[spectra_range['R5']][-1]-10)  
                
    #         if chisq_val_R6 >= 0.05:
    #             if count(ga_solution[:num_oscillators], v[spectra_range['R6']]) < max_num_oscillators_R6:
    #                 lb_ga_nu = np.append(lb_ga_nu, v[spectra_range['R6']][0]+10)
    #                 ub_ga_nu = np.append(ub_ga_nu, v[spectra_range['R6']][-1]-50) 

        
        num_oscillators = len(lb_ga_nu)
        
        lb_ga_nu = np.sort(lb_ga_nu)
        ub_ga_nu = np.sort(ub_ga_nu)

        lb_ga_gamm = np.repeat(0, num_oscillators)
        ub_ga_gamm = np.repeat(2, num_oscillators)

        lb_ga_fourpr = np.repeat(0, num_oscillators)
        ub_ga_fourpr = np.repeat(2, num_oscillators)

        lb_ga = np.hstack((lb_ga_nu, lb_ga_gamm, lb_ga_fourpr)) #lower bound of genes representing gamm, fourpr
        ub_ga = np.hstack((ub_ga_nu, ub_ga_gamm, ub_ga_fourpr)) #upper bound of genes representing gamm, fourpr

        gene_space = []
        for i in np.arange(len(lb_ga)):
            gene_space.append({'low': lb_ga[i], 'high': ub_ga[i]})
        gene_space.append({'low':1, 'high':4})


        ga_instance = ga_optical_constants(gene_space)
        ga_solution, ga_solution_fitness, ga_solution_idx = ga_instance.best_solution()
    #     ga_instance.plot_fitness()

        R_ga,n_ga,k_ga = calculate_rnk(ga_solution, v, viewing_angle)


        fig, ax = plt.subplots(1,4, figsize=(15, 3))
        ax[0].plot(v,n_ga,'r', label='n_ga')
        ax[0].legend(bbox_to_anchor=(1,1))
        ax[0].set_title('GA modelled n')

        ax[1].plot(v,k_ga,'g', label='k_ga')
        ax[1].legend(bbox_to_anchor=(1,1))
        ax[1].set_title('GA modelled k')

        ax[2].plot(v,R_ga,'b', label='R_ga')
        ax[2].plot(v,R_smoothed,'k', label='R')
        ax[2].legend(bbox_to_anchor=(1,1))
        ax[2].set_title('GA modelled R')  

        ax[3].plot(v, R_smoothed-R_ga, label='residual R')
        ax[3].set_ylim(-0.05, +0.05)
        plt.show()

        print("\t".join(map(str, ga_solution)))
        

        print("Fitness value of the best solution = {solution_fitness}".format(solution_fitness=ga_solution_fitness))

        if ga_instance.best_solution_generation != -1:
            print("Best fitness value reached after {best_solution_generation} generations.".format(best_solution_generation=ga_instance.best_solution_generation))

        chisq_val = chisq(R_ga, R_smoothed)
        print("Chi-square goodness of overall fitness is ", chisq_val)

        chisq_val_R1 = chisq(R_ga[spectra_range['R1']], R_smoothed[spectra_range['R1']])
        chisq_val_R2 = chisq(R_ga[spectra_range['R2']], R_smoothed[spectra_range['R2']])
        chisq_val_R3 = chisq(R_ga[spectra_range['R3']], R_smoothed[spectra_range['R3']])
        chisq_val_R4 = chisq(R_ga[spectra_range['R4']], R_smoothed[spectra_range['R4']])
        chisq_val_R5 = chisq(R_ga[spectra_range['R5']], R_smoothed[spectra_range['R5']])
        chisq_val_R6 = chisq(R_ga[spectra_range['R6']], R_smoothed[spectra_range['R6']])
        
        print("Chi-square goodness of R1 fitness is ", chisq_val_R1)
        print("Chi-square goodness of R2 fitness is ", chisq_val_R2)
        print("Chi-square goodness of R3 fitness is ", chisq_val_R3)
        print("Chi-square goodness of R4 fitness is ", chisq_val_R4)
        print("Chi-square goodness of R5 fitness is ", chisq_val_R5)
        print("Chi-square goodness of R6 fitness is ", chisq_val_R6)
        

    #     np.savetxt('oscillation_parameters_new.txt', ga_solution)
        np.savetxt('oscillation_params/run%s_%s_NumOfOscillators_ga_oscillation_parameters.txt'%(run, num_oscillators), ga_solution)
        np.savetxt("oscillation_params/run%s_%s_NumOfOscillators_ga_chi_sq.txt"%(run, num_oscillators), np.stack((chisq_val, chisq_val_R1, chisq_val_R2, chisq_val_R3, chisq_val_R4, chisq_val_R5, chisq_val_R6)), header="overall_chisq, R1_chisq, R2_chisq, R3_chisq, R4_chisq, R5_chisq, R6_chisq")
        np.savetxt("optical_constants/run%s_%s_NumOfOscillators_ga_optical_constants.txt"%(run, num_oscillators), np.stack((v, R_ga, R_smoothed-R_ga, n_ga, k_ga), axis=1), header="wavenumber, R_modelled, R_residuals, n , k")
        np.savetxt('best_solution_fitness/run%s_%s_NumOfOscillators_ga_best_solution_fitness_per_generation.txt'%(run, num_oscillators), ga_instance.best_solutions_fitness)
        
        
        
        ga_oscillation_parameters = np.copy(ga_solution)

        N = int((len(ga_oscillation_parameters)-1)/3)

        nu=ga_oscillation_parameters[0:N]
        gamm=ga_oscillation_parameters[N:N+N]
        fourpr=ga_oscillation_parameters[N+N:N+N+N]
        epsil=ga_oscillation_parameters[N+N+N] 

        ga_oscillators_initial = np.hstack((nu,gamm,fourpr,epsil))

        # gamm=gamm./nu;
        # fourpr=Sk./(nu.^2);

        # Set lower and upper bounds
        lb_nu=-20.+nu
        ub_nu=20.+nu
        lb_gamm=np.zeros((1,len(nu)))
        ub_gamm=np.tile(4,(1,len(nu)))
        lb_fourpr=np.zeros((1,len(nu)))
        ub_fourpr=np.tile(4,(1,len(nu)))


        lb=np.hstack((lb_nu,lb_gamm.flatten(),lb_fourpr.flatten(),1))
        ub=np.hstack((ub_nu,ub_gamm.flatten(),ub_fourpr.flatten(),4))

        ga_oscillators_final, lsq_cov = curve_fit(func_lsq_R, xdata=v, p0=ga_oscillators_initial, ydata=R_smoothed, bounds=(lb,ub), maxfev=10000)

        R_final, n_final, k_final = calculate_rnk(ga_oscillators_final,v, viewing_angle)
        
        N = int((len(ga_oscillators_final)-1)/3)

        nu=ga_oscillators_final[0:N]
        gamm=ga_oscillators_final[N:N+N]
        fourpr=ga_oscillators_final[N+N:N+N+N]
        epsil=ga_oscillators_final[N+N+N] 
        

        fig, ax = plt.subplots(1,4, figsize=(15, 3))
        ax[0].plot(v, n_ga, 'g', label='n_ga')
        ax[0].plot(v, n_final, 'b', label='n_ga+lsq')
        ax[0].legend(bbox_to_anchor=(1,1))
        ax[0].set_title('GA+LSQ modelled n')

        ax[1].plot(v,k_ga,'g', label='k_ga')
        ax[1].plot(v,k_final,'b', label='k_ga+lsq')
        ax[1].legend(bbox_to_anchor=(1,1))
        ax[1].set_title('GA+LSQ modelled k')

        ax[2].plot(v,R_final,'b', label='R_ga+lsq')
        ax[2].plot(v,R_smoothed,'k', label='R')
        ax[2].legend(bbox_to_anchor=(1,1))
        ax[2].set_title('GA+LSQ modelled R')  

        ax[3].plot(v, R_smoothed-R_final, label='residual R')
        ax[3].set_ylim(-0.05, +0.05)
        plt.legend()
        plt.show()

        chisq_val = chisq(R_final[1:], R_smoothed[1:])

        print("Chi-square goodness of fitness is ", chisq_val)

        chisq_val_R1 = chisq(R_final[spectra_range['R1']], R_smoothed[spectra_range['R1']])
        chisq_val_R2 = chisq(R_final[spectra_range['R2']], R_smoothed[spectra_range['R2']])
        chisq_val_R3 = chisq(R_final[spectra_range['R3']], R_smoothed[spectra_range['R3']])
        chisq_val_R4 = chisq(R_final[spectra_range['R4']], R_smoothed[spectra_range['R4']])
        chisq_val_R5 = chisq(R_final[spectra_range['R5']], R_smoothed[spectra_range['R5']])
        chisq_val_R6 = chisq(R_final[spectra_range['R6']], R_smoothed[spectra_range['R6']])
        
        print("Chi-square goodness of R1 fitness is ", chisq_val_R1)
        print("Chi-square goodness of R2 fitness is ", chisq_val_R2)
        print("Chi-square goodness of R3 fitness is ", chisq_val_R3)
        print("Chi-square goodness of R4 fitness is ", chisq_val_R4)
        print("Chi-square goodness of R5 fitness is ", chisq_val_R5)
        print("Chi-square goodness of R6 fitness is ", chisq_val_R6)
        
        
        fig, ax2 = plt.subplots(1,2, figsize=(10, 3))

        ax2[0].plot(v,R_smoothed)
        for i in ga_solution[:num_oscillators]:
            ax2[0].scatter(i, R_smoothed[np.abs(v-i).argmin()], label=round(i,3), color='r')
        ax2[0].legend()
        ax2[0].set_title("GA oscillators")

        ax2[1].plot(v,R_smoothed)
        for i in ga_oscillators_final[:num_oscillators]:
            ax2[1].scatter(i, R[np.abs(v-i).argmin()], label=round(i,3), color='r')
        ax2[1].legend()
        ax2[1].set_title("GA+LSQ oscillators")
        plt.show()

        

        R_residuals = R_smoothed-R_final
        
        np.savetxt('oscillation_params/run%s_%s_NumOfOscillators_lsq_oscillation_parameters.txt'%(run, num_oscillators), ga_oscillators_final)
        np.savetxt("optical_constants/run%s_%s_NumOfOscillators_lsq_optical_constants.txt"%(run, num_oscillators), np.stack((v, R_final, R_residuals, n_final, k_final), axis=1), header="wavenumber, R_modelled, R_residuals, n , k")
        np.savetxt("oscillation_params/run%s_%s_NumOfOscillators_lsq_chi_sq.txt"%(run, num_oscillators), np.stack((chisq_val, chisq_val_R1, chisq_val_R2, chisq_val_R3, chisq_val_R4, chisq_val_R5, chisq_val_R6)), header="overall_chisq, R1_chisq, R2_chisq, R3_chisq, R4_chisq, R5_chisq, R6_chisq")
        
        run+=1
        
        if (N>=total_num_oscillators) or (run>=total_num_oscillators):
            break






    



Loading BokehJS ...

  0%|          | 0/10000 [00:00<?, ?it/s]name 'viewing_angle' is not defined
Traceback (most recent call last):
  File "/Users/mrecarte/Library/Python/3.9/lib/python/site-packages/pygad/pygad.py", line 1669, in cal_pop_fitness
    fitness = self.fitness_func(self, sol, sol_idx)
  File "/var/folders/r0/gwt02kcx3jn5lm9hqbp5nj7w0000gq/T/ipykernel_81640/667647128.py", line 161, in fitness_func
    r, n, k = calculate_rnk(solution, v, viewing_angle)
NameError: name 'viewing_angle' is not defined
  0%|          | 0/10000 [00:00<?, ?it/s]


AttributeError: 'tuple' object has no attribute 'tb_frame'